In [1]:
# Libraries import
import sys
import os
import os.path as pth
sys.path.insert(0, '../..')
    
from lcav.lca_problem import *
from lcav.helpers import *
from lcav.io.configuration import *
from lcav.gui.plots import *
import matplotlib.pyplot as plt 
from premise import clear_cache

# Pretty print for Sympy
from sympy import init_printing
init_printing()

# Paths
#os.chdir('./notebook')
DATA_FOLDER = './data'
CONFIGURATION_FILE = pth.join(DATA_FOLDER, 'configuration_methodo_ei391.yaml')

#clear_cache()  # clear premise cache

In [18]:
#agb.findActivity('kerosene production, synthetic, Fischer Tropsch process, hydrogen from electrolysis, energy allocation', db_name='ecoinvent_cutoff_3.9_remind_SSP2-Base_2050', loc='World').listExchanges()
agb.findActivity('kerosene production, synthetic, Fischer Tropsch process, hydrogen from electrolysis, energy allocation\n(remind_SSP2-Base_2020)', 
                 loc='World',
                 db_name = 'Foreground DB').listExchanges()

[('Heat, waste',
  'Heat, waste' (megajoule, None, ('air',)),
  0.6554884173913044),
 ('Water', 'Water' (cubic meter, None, ('water',)), 0.00019358984347826041),
 ('Carbon dioxide, non-fossil',
  'Carbon dioxide, non-fossil' (kilogram, None, ('air',)),
  0.22),
 ('gas-to-liquid plant construction',
  'gas-to-liquid plant construction' (unit, GLO, None),
  6.7e-12),
 ('market group for electricity, low voltage',
  'electricity production, photovoltaic, commercial' (kilowatt hour, FR, None),
  0.000342925964905733),
 ('market group for electricity, low voltage',
  'electricity production, photovoltaic, commercial' (kilowatt hour, FR, None),
  0.000342925964905733),
 ('market group for electricity, low voltage',
  'electricity production, photovoltaic, commercial' (kilowatt hour, FR, None),
  0.000342925964905733),
 ('market group for electricity, low voltage',
  'electricity production, photovoltaic, commercial' (kilowatt hour, FR, None),
  0.000342925964905733),
 ('market group for elec

In [3]:
_, model, methods = LCAProblemConfigurator(CONFIGURATION_FILE).generate()

Initial setup of EcoInvent/Premise already done, skipping. To reset the project use option `reset=True`.
Building LCA model from configuration file... Done.


In [4]:
process_tree(model, foreground_only=False, colormap='Pastel2')

In [5]:
agb.list_databases()

,backend,nb_activities,type
name,,,
biosphere3,sqlite,4721,biosphere
ecoinvent-3.9.1-cutoff,sqlite,21238,background
ecoinvent_cutoff_3.9_remind_SSP2-Base_2020,sqlite,28724,background
ecoinvent_cutoff_3.9_remind_SSP2-Base_2025,sqlite,28794,background
ecoinvent_cutoff_3.9_remind_SSP2-Base_2030,sqlite,29228,background
ecoinvent_cutoff_3.9_remind_SSP2-Base_2035,sqlite,29550,background
ecoinvent_cutoff_3.9_remind_SSP2-Base_2040,sqlite,29816,background
ecoinvent_cutoff_3.9_remind_SSP2-Base_2045,sqlite,29942,background
ecoinvent_cutoff_3.9_remind_SSP2-Base_2050,sqlite,30068,background


In [6]:
agb.list_parameters()

group,name,label,default,min,max,std,distrib,unit,db
,energy_consumption_electrofuel,energy consumption electrofuel,1.0,1,1,,linear,,Foreground DB
,model,model,remind,,,,,,Foreground DB
,pathway,pathway,SSP2_Base,,,,,,Foreground DB
,year,year,2050,2020,2050,,linear,,Foreground DB


In [7]:
list_processes(model, foreground_only=True)

,activity,unit,location,level,database,parent,exchange
model,model,None,GLO,0,Foreground DB,,
fuel,fuel,MJ,GLO,1,Foreground DB,model,1.00
efuel,efuel,kilogram,GLO,2,Foreground DB,fuel,0.0231481481481481*energy_consumption_electrofuel
production,production,kilogram,GLO,3,Foreground DB,efuel,1.00
"kerosene production, synthetic, Fischer Tropsch process, hydrogen from electrolysis, energy allocation\n(remind_SSP2-Base)","kerosene production, synthetic, Fischer Tropsc...",kilogram,GLO,4,Foreground DB,production,model_remind*pathway_SSP2_Base
"kerosene production, synthetic, Fischer Tropsch process, hydrogen from electrolysis, energy allocation\n(remind_SSP2-Base_2020) [World]","kerosene production, synthetic, Fischer Tropsc...",kilogram,World,5,Foreground DB,"kerosene production, synthetic, Fischer Tropsc...",205/3 - year/30
syngas_updated,syngas_updated,None,GLO,6,Foreground DB,"kerosene production, synthetic, Fischer Tropsc...",2.35
syngas_photovolatic,syngas_photovolatic,kilogram,GLO,7,Foreground DB,syngas_updated,1.00
"syngas, RWGS, Production, for Fischer Tropsch process, hydrogen from electrolysis\n(remind_SSP2-Base)","syngas, RWGS, Production, for Fischer Tropsch ...",kilogram,GLO,8,Foreground DB,syngas_photovolatic,model_remind*pathway_SSP2_Base
"syngas, RWGS, Production, for Fischer Tropsch process, hydrogen from electrolysis\n(remind_SSP2-Base_2020) [World]","syngas, RWGS, Production, for Fischer Tropsch ...",kilogram,World,9,Foreground DB,"syngas, RWGS, Production, for Fischer Tropsch ...",205/3 - year/30


In [8]:
parameters = {
    'year': 2020,
    'model': 'remind',
    'pathway': 'SSP2_Base',
    'energy_consumption_electrofuel': 10e6,  
}

res = agb.compute_impacts(
    model,
    methods,
    return_params=False,
    axis='phase',
    **parameters,
)
res

,acidification: terrestrial - terrestrial acidification potential (TAP)[kg SO2-Eq],climate change - global warming potential (GWP100)[kg CO2-Eq],"ecotoxicity: freshwater - freshwater ecotoxicity potential (FETP)[kg 1,4-DCB-Eq]","ecotoxicity: marine - marine ecotoxicity potential (METP)[kg 1,4-DCB-Eq]","ecotoxicity: terrestrial - terrestrial ecotoxicity potential (TETP)[kg 1,4-DCB-Eq]","energy resources: non-renewable, fossil - fossil fuel potential (FFP)[kg oil-Eq]",eutrophication: freshwater - freshwater eutrophication potential (FEP)[kg P-Eq],eutrophication: marine - marine eutrophication potential (MEP)[kg N-Eq],"human toxicity: carcinogenic - human toxicity potential (HTPc)[kg 1,4-DCB-Eq]","human toxicity: non-carcinogenic - human toxicity potential (HTPnc)[kg 1,4-DCB-Eq]",...,land use - agricultural land occupation (LOP)[m2*a crop-Eq],material resources: metals/minerals - surplus ore potential (SOP)[kg Cu-Eq],ozone depletion - ozone depletion potential (ODPinfinite)[kg CFC-11-Eq],particulate matter formation - particulate matter formation potential (PMFP)[kg PM2.5-Eq],photochemical oxidant formation: human health - photochemical oxidant formation potential: humans (HOFP)[kg NOx-Eq],photochemical oxidant formation: terrestrial ecosystems - photochemical oxidant formation potential: ecosystems (EOFP)[kg NOx-Eq],water use - water consumption potential (WCP)[m3],total: ecosystem quality - ecosystem quality[species.yr],total: human health - human health[DALYs],total: natural resources - natural resources[USD 2013]
phase,,,,,,,,,,,,,,,,,,,,,
_other_,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
production_electrofuel,"10,769.3",2.8932e+06,"365,771","480,856",3.48973e+07,"846,208","1,532.26",188.987,"334,877",5.45406e+06,...,"347,462","70,030",0.784211,"3,763.01","5,369.47","5,665.82","32,196.1",0.0159293,7.4829,"236,233"
*sum*,"10,769.3",2.8932e+06,"365,771","480,856",3.48973e+07,"846,208","1,532.26",188.987,"334,877",5.45406e+06,...,"347,462","70,030",0.784211,"3,763.01","5,369.47","5,665.82","32,196.1",0.0159293,7.4829,"236,233"


In [204]:
parameters = {
    'year': [year for year in range(2020, 2060)],
    'model': 'remind',
    'pathway': 'SSP2_Base'
}

res = agb.compute_impacts(
    model,
    methods,
    return_params=False,
    **parameters,
)
res

,acidification - accumulated exceedance (AE)[mol H+-Eq],climate change - global warming potential (GWP100)[kg CO2-Eq],water use - user deprivation potential (deprivation-weighted water consumption)[m3 world eq. deprived],land use - soil quality index[dimensionless]
year,,,,
2020,5.50573e+07,1.47135e+10,4.65833e+09,1.86542e+11
2021,5.43462e+07,1.44646e+10,4.61084e+09,1.88522e+11
2022,5.36351e+07,1.42157e+10,4.56334e+09,1.90503e+11
2023,5.2924e+07,1.39668e+10,4.51585e+09,1.92484e+11
2024,5.22129e+07,1.37179e+10,4.46836e+09,1.94464e+11
2025,5.15018e+07,1.3469e+10,4.42086e+09,1.96445e+11
2026,5.07907e+07,1.32201e+10,4.37337e+09,1.98426e+11
2027,5.00796e+07,1.29712e+10,4.32588e+09,2.00407e+11
2028,4.93685e+07,1.27223e+10,4.27839e+09,2.02387e+11


In [209]:
parameters = {
    'year': [year for year in range(2020, 2051)],
    'share_efuel': [i/2051 for i in range(2020, 2051)],
    'model': 'remind',
    'pathway': 'SSP2_Base'
}

# Create an empty DataFrame to store the results for each impact method and each year
multi_df_lca = pd.DataFrame()

for i, year in enumerate(parameters['year']):
    parameters_tmp = parameters.copy()
    for key, val in parameters_tmp.items():
        if isinstance(val, list):
            parameters_tmp[key] = val[i]

    res = agb.compute_impacts(
        model,
        methods,
        axis='phase',
        **parameters_tmp,
    )
    
    # Iterate over each method
    df_year = pd.DataFrame()  # DataFrame for the results of each impact method for the current year
    for method in res.columns:
        # Extract the data for the current method
        data = res[method]
        # Create a DataFrame with MultiIndex consisting of method and year
        df_year_method = pd.DataFrame(data.values, columns=[year], index=pd.MultiIndex.from_product([[method], data.index], names=['method', 'phase']))
        # Concatenate the new DataFrame with the existing DataFrame
        df_year = pd.concat([df_year, df_year_method], axis=0)
        
    multi_df_lca = pd.concat([multi_df_lca, df_year], axis=1)

multi_df_lca

2020  \
method                                             phase                             
acidification - accumulated exceedance (AE)[mol... _other_              0.00937195   
                                                   aircraft_production      47,120   
                                                   airport             5.50102e+07   
                                                   combustion           0.00017518   
                                                   kerosene_production 5.73896e-05   
                                                   *sum*               5.50573e+07   
climate change - global warming potential (GWP1... _other_                 12.0572   
                                                   aircraft_production 7.30175e+06   
                                                   airport             1.47062e+10   
                                                   combustion            0.0481539   
                                                   kerosene_production   0.0128541   
                                                   *sum*               1.47135e+10   
water use - user deprivation potential (depriva... _other_                0.450191   
                                                   aircraft_production 1.56423e+06   
                                                   airport             4.65677e+09   
                                                   combustion          0.000798481   
                                                   kerosene_production 0.000948419   
                                                   *sum*               4.65833e+09   
land use - soil quality index[dimensionless]       _other_                 15.4547   
                                                   aircraft_production 1.80652e+07   
                                                   airport             1.86524e+11   
                                                   combustion                    0   
                                                   kerosene_production   0.0385319   
                                                   *sum*               1.86542e+11   

                                                                              2021  \
method                                             phase                             
acidification - accumulated exceedance (AE)[mol... _other_              0.00932151   
                                                   aircraft_production    46,889.5   
                                                   airport             5.42993e+07   
                                                   combustion          0.000169529   
                                                   kerosene_production  5.5468e-05   
                                                   *sum*               5.43462e+07   
climate change - global warming potential (GWP1... _other_                 12.0455   
                                                   aircraft_production 7.26017e+06   
                                                   airport             1.44574e+10   
                                                   combustion            0.0466006   
                                                   kerosene_production   0.0124203   
                                                   *sum*               1.44646e+10   
water use - user deprivation potential (depriva... _other_                0.447259   
                                                   aircraft_production 1.55616e+06   
                                                   airport             4.60928e+09   
                                                   combustion          0.000772723   
                                                   kerosene_production 0.000913752   
                                                   *sum*               4.61084e+09   
land use - soil quality index[dimensionless]       _other_                 15.5327   
                                   

In [221]:
test = get_parameter('aircraft_lifetime')
test.is_aeromaps_output = True
test.

In [222]:
test = list()
test += ['a', 'b']
test

['a', 'b']

In [196]:
# Assuming df is your DataFrame
series_list = []

# Iterate over each row of the DataFrame
for index, row in multi_df_lca.iterrows():
    # Convert the row to a Pandas Series
    series = row.squeeze()
    # Set the name of the series as the tuple index
    series.name = index
    # Append the series to the list
    series_list.append(series)

# Now series_list contains all the Pandas Series corresponding to each row of the MultiIndex DataFrame
#series_list

In [212]:
multi_df_lca.columns

Index([2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031,
       2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043,
       2044, 2045, 2046, 2047, 2048, 2049, 2050],
      dtype='int64')

In [215]:
test = pd.DataFrame(columns=list(range(2020, 2051)))
test.columns

Index([2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031,
       2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043,
       2044, 2045, 2046, 2047, 2048, 2049, 2050],
      dtype='int64')